In [1]:
from ipywidgets import widgets
from IPython.display import display, Markdown, clear_output
from ipywidgets import HTML
import pandas as pd
import numpy as np


df = pd.read_csv('verbs.csv')
categories = np.array(['presente', 'imperfecto', 
                       'pretérito', 'futuro', 'condicional'])

In [2]:
class SpanishApp:
    
    def __init__(self):
        self._initialize_attributes()
        self._create_widgets()
        
        self.set_question_visibility('hidden')
        self.set_answer_visibility('hidden')
        
        self.search_btn.on_click(self.search_new_verb)
        self.check_answer_btn.on_click(self.check_answer)
        
    ###
    # Constructors
    ###
    def _initialize_attributes(self):
        self.conjugated_verb = None
        self.infinitive = None
        self.conjugated_category = None
        self.include_vos = False
    
    def _create_widgets(self):
        self.html_title = HTML("""<h1><center><u>Spanish conjugation tool</u></center></h1><h3><center>Similarity search engine</center></h3>""")
        self.hline = HTML("<hline>")
        self.html2 = HTML("""<br><h3><center>Choose what conjugations to include</center></h3>""")
        self.base_word_html = HTML(f"<p>{self.infinitive}, {self.conjugated_category}")
        self.answer_html = HTML(f"{self.conjugated_verb}")
        self.answer_indicator = widgets.HTML()
        
        self.cat_selection = widgets.SelectMultiple(options=categories)
        self.answer_input = widgets.Text(description='Answer:')
        
        self.search_btn = widgets.Button(description='Search verb')
        self.check_answer_btn = widgets.Button(description='Check answer')
        
        self.verb_output = widgets.Output()
        self.title_output = widgets.Output()
        
        vbox_layout = widgets.Layout(display='flex',
                               flex_flow='column',
                               align_items='center',
                               width='100%')
        
        check_answer_box = widgets.HBox(children=[self.answer_input, 
                                                  self.answer_indicator])
        check_answer_btn_box = widgets.HBox(children=[self.search_btn, 
                                                      self.check_answer_btn])
        
        self.box = widgets.VBox(children=
                                [
                                    self.html_title, self.hline, self.html2, self.hline,
                                    self.cat_selection, check_answer_btn_box,
                                    self.base_word_html, check_answer_box,
                                    self.answer_html,
                                ],
                               layout=vbox_layout)


    ###
    # Updates
    ###
    def update_answer(self, value):
        self.conjugated_verb = value
        self._update_answer_html()
        
    def update_infinitive(self, value):
        self.infinitive = value
        self._update_baseword()
        
    def update_conjugated_category(self, value):
        self.conjugated_category = value
        self._update_baseword()
        
    def _update_baseword(self):
        self.base_word_html.value = f"<p><b>{self.infinitive}</b> (<i>{self.conjugated_category}</i>)"
    
    ###
    # Setters
    ###
    def set_question_visibility(self, visibility):
        self.base_word_html.layout.visibility = visibility
        self.check_answer_btn.layout.visibility = visibility
        self.answer_input.layout.visibility = visibility
        self.answer_input.value = ""
        self.answer_indicator.value = ''
    
    def set_answer_visibility(self, visibility):
        self.answer_html.layout.visibility = visibility
        self.answer_indicator.visibility = visibility
        
    def _update_answer_html(self):
        self.answer_html.value = f"<b>{self.conjugated_verb}</b>"
        
        
    ###
    # Eventlisteners
    ###
    def search_new_verb(self, b):
        self.set_answer_visibility('hidden')
        
        verb = np.random.choice(df.verb, 1, p=df.prob)
        row_chosen = df.verb.isin(verb)
        # Take all true values possible
        categories_chosen = np.sum([df.columns.str.contains(x) for x in self.cat_selection.value], 
                                   axis=0, dtype=bool)
        
        cat_str_chosen = df.columns[categories_chosen].values

        # Remove vosotros
        if not self.include_vos:
            cat_str_chosen = [x for x in cat_str_chosen if 'vos' not in x]
        
        # Choose random verb to be displayed
        cat_to_conj = np.random.choice(cat_str_chosen, 1)[0]

        conjugated_verb = df.loc[row_chosen, df.columns.isin([cat_to_conj])].values[0]
        
        self.update_answer(str(conjugated_verb[0]))
        self.update_infinitive(str(verb[0]))
        self.update_conjugated_category(cat_to_conj)
        self.set_question_visibility('visible')
    

    def check_answer(self, b):
        answer = self.answer_input.value
        if answer == self.conjugated_verb:
            self.answer_indicator.value = 'Correct'
            
        else:
            self.answer_indicator.value = 'Incorrect'
        self.set_answer_visibility('visible')
        
app = SpanishApp()
display(app.box)

C:\Users\teemu\anaconda3\envs\datascience\lib\site-packages\traitlets\traitlets.py:586: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)


NameError: name 'include_vos' is not defined